# Extended ECCE paper code
* tested in conda python 3.10 environment (torchvision that is needed for pytorch_widedeep is not supported for 3.11 as of 26th Jan 2023)
```
conda create -n ecce python=3.10
pip install -r requirements.txt
```

## Imports

In [1]:
import sys
import pandas as pd

# to save results to data directory
module_path = ".."
if module_path not in sys.path:
    sys.path.insert(1, module_path)
# increase displayed columns in jupyter notebook
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

In [2]:
import dill
import numpy as np
import matplotlib.pyplot as plt

import h2o
from h2o.automl import H2OAutoML

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pytorch_widedeep.dataloaders import DataLoaderImbalanced
from pytorch_widedeep.models import TabTransformer, WideDeep
from pytorch_widedeep.training import Trainer
from pytorch_widedeep.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LRHistory,
    Callback,
)
from pytorch_widedeep.initializers import (
    XavierNormal,
)
from torch.optim import Adam, SGD, lr_scheduler#, NAdam

from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune import JupyterNotebookReporter
from ray.tune.integration.wandb import WandbLoggerCallback, wandb_mixin
import wandb

import src.utils as utils
from src import pytorch_widedeep_explainer
from src import lightgbm_explainer
import src.common as common

import lightgbm as lgbm

import tracemalloc
import warnings
from typing import Optional, Dict
tracemalloc.start()

class RayTuneReporter(Callback):
    r"""Callback that allows reporting history and lr_history values to RayTune
    during Hyperparameter tuning

    Callbacks are passed as input parameters to the ``Trainer`` class. See
    :class:`pytorch_widedeep.trainer.Trainer`

    For examples see the examples folder at:

        .. code-block:: bash

            /examples/12_HyperParameter_tuning_w_RayTune.ipynb
    """

    def on_epoch_end(
        self, epoch: int, logs: Optional[Dict] = None, metric: Optional[float] = None
    ):
        report_dict = {}
        for k, v in self.trainer.history.items():
            report_dict.update({k: v[-1]})
        if hasattr(self.trainer, "lr_history"):
            for k, v in self.trainer.lr_history.items():
                report_dict.update({k: v[-1]})
        tune.report(report_dict)


class WnBReportBest(Callback):
    r"""Callback that allows reporting best performance of a run to WnB
    during Hyperparameter tuning. It is an adjusted pytorch_widedeep.callbacks.ModelCheckpoint
    with added WnB and removed checkpoint saving.

    Callbacks are passed as input parameters to the ``Trainer`` class.

    Parameters
    ----------
    wb: obj
        Weights&Biases API interface to report single best result usable for
        comparisson of multiple paramater combinations by, for example,
        `parallel coordinates
        <https://docs.wandb.ai/ref/app/features/panels/parallel-coordinates>`_.
        E.g W&B summary report `wandb.run.summary["best"]`.
    monitor: str, default="loss"
        quantity to monitor. Typically `'val_loss'` or metric name
        (e.g. `'val_acc'`)
    mode: str, default="auto"
        If ``save_best_only=True``, the decision to overwrite the current save
        file is made based on either the maximization or the minimization of
        the monitored quantity. For `'acc'`, this should be `'max'`, for
        `'loss'` this should be `'min'`, etc. In `'auto'` mode, the
        direction is automatically inferred from the name of the monitored
        quantity.

    """

    def __init__(
        self,
        wb: object,
        monitor: str = "val_loss",
        mode: str = "auto",
    ):
        super(WnBReportBest, self).__init__()

        self.monitor = monitor
        self.mode = mode
        self.wb = wb

        if self.mode not in ["auto", "min", "max"]:
            warnings.warn(
                "WnBReportBest mode %s is unknown, "
                "fallback to auto mode." % (self.mode),
                RuntimeWarning,
            )
            self.mode = "auto"
        if self.mode == "min":
            self.monitor_op = np.less
            self.best = np.Inf
        elif self.mode == "max":
            self.monitor_op = np.greater  # type: ignore[assignment]
            self.best = -np.Inf
        else:
            if self._is_metric(self.monitor):
                self.monitor_op = np.greater  # type: ignore[assignment]
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_epoch_end(  # noqa: C901
        self, epoch: int, logs: Optional[Dict] = None, metric: Optional[float] = None
    ):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is not None:
            if self.monitor_op(current, self.best):
                self.wb.run.summary["best"] = current  # type: ignore[attr-defined]
                self.best = current
                self.best_epoch = epoch

    @staticmethod
    def _is_metric(monitor: str):
        "copied from pytorch_widedeep.callbacks"
        if any([s in monitor for s in ["acc", "prec", "rec", "fscore", "f1", "f2"]]):
            return True
        else:
            return False


/home/palo/miniconda3/envs/ecce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


# Dataset

**identifiers**

In [3]:
column_types = common.json_load("../#datasets/Colab_PowerConverter/column_types.json")
target = column_types["target"]
identifier = column_types["identifier"]
measurement_label = column_types["measurement_label"]

parameters = {
    "random_state": 1,
    "valid_size": 0.2,
    "test_size": 0.5,
    "scaler_mapper_def": {
        "target_col": None,
        "identifier_col": None,
        "cont_cols": StandardScaler,
    },
}

valid_size = parameters["valid_size"]
test_size = parameters["test_size"]
scaler_mapper_def = parameters["scaler_mapper_def"]
random_state = parameters["random_state"]
test_n_valid_combined = True
task = "multiclass" #(or "binary")

In [4]:
df = pd.read_pickle("../#datasets/Colab_PowerConverter/dataset.pkl")
# df = pd.read_csv("../#datasets/Colab_PowerConverter/PowerElectronics_11.csv")
# df = pd.read_csv("../#datasets/Colab_PowerConverter/PowerElectronics_11_tampered_1_10.csv")
# df = pd.read_csv("../#datasets/Colab_PowerConverter/PowerElectronics_11_tampered_1_10_reduced.csv")
# df = pd.read_csv("../#datasets/Colab_PowerConverter/PowerElectronics_tampered_1_100.csv")
# df = pd.read_csv("../#datasets/Colab_PowerConverter/PowerElectronics_tampered_1_100_reduced.csv")

**Ioannis datasets**

In [5]:
datasets = [
    "../#datasets/Colab_PowerConverter/PowerElectronics_11.csv",
    "../#datasets/Colab_PowerConverter/PowerElectronics_11_tampered_1_10.csv",
    "../#datasets/Colab_PowerConverter/PowerElectronics_11_tampered_1_10_reduced.csv",
    "../#datasets/Colab_PowerConverter/PowerElectronics_tampered_1_100.csv",
    "../#datasets/Colab_PowerConverter/PowerElectronics_tampered_1_100_reduced.csv",
    ]
for dataset_str in datasets:
    df_temp = pd.read_csv(dataset_str)
    print(df_temp["Target"].value_counts())
df_temp.head()

0    14000
2     5000
1     4000
3     1000
Name: Target, dtype: int64
0    14010
2     5000
1     3990
3     1000
Name: Target, dtype: int64
0    11653
2     3878
1     2236
3      693
Name: Target, dtype: int64
 2    5000
 1    3901
 0    2099
-1    1000
Name: Target, dtype: int64
 1    2120
 2    1939
 0     407
-1     252
Name: Target, dtype: int64


,URa,URb,URc,Udc,IRa,IRb,IRc,Target
0,-307.438,101.4930,205.946,599.277,0.300602,-1.295490,0.134279,0
1,-303.949,85.6335,218.316,601.601,0.416571,0.805675,-0.946058,0
2,-259.495,-21.7492,281.244,603.314,0.759898,-0.027466,-0.462348,0
3,-235.802,-59.0828,294.885,603.069,1.112380,0.785838,-0.277714,0
4,-221.591,-81.6662,303.257,601.846,0.384527,-0.579842,0.090028,0


**Keep only the faulty datasets as the other ones do not include fault**

In [6]:
# this measurement did not have a fault (?)
# df = df[df[measurement_label]!="Single-Phase_Sensor_Fault"]
# df.reset_index(inplace=True, drop=True)

df = df[df[measurement_label].isin(["LL_Fault", "Three-Phase_Sensor_Fault", "Single_Phase_Sag", "Three_Phase_Grid_Fault"])]
df.reset_index(inplace=True, drop=True)

fault_dict = {}
for label,i in zip(df[measurement_label].unique(), range(len(df[measurement_label].unique()))):
    df.loc[(df[measurement_label]==label) & (df[target]==1), target] = int(i+1)
    fault_dict[label] = int(i+1)

print(f"imbalance of the classes:\n{df[target].value_counts()}")
print(f"faults dictionary:\n{fault_dict}")

imbalance of the classes:
0    235539
2     40014
3     40001
1      3166
4      1335
Name: fault, dtype: int64
faults dictionary:
{'LL_Fault': 1, 'Three-Phase_Sensor_Fault': 2, 'Single_Phase_Sag': 3, 'Three_Phase_Grid_Fault': 4}


# Preprocessing
* divide dataset into train, test, valid
* scale continuous columns by standard scaler(not needed for LightGBM but for other mdoels...)

In [7]:
df.drop(columns=[measurement_label], inplace=True)

df_train, df_valid = train_test_split(df, test_size=valid_size, stratify=df[target], random_state=random_state)
df_valid, df_test = train_test_split(df_valid, test_size=test_size, stratify=df_valid[target], random_state=random_state)

df_train.reset_index(inplace=True, drop=True)
df_valid.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

cont_cols = df.drop(columns=[target,identifier]).columns.values
scaler = utils.scaler_mapper(
    cont_cols=cont_cols,
    target_col=target,
    identifier=identifier,
    scaler_mapper_def=scaler_mapper_def,
)

df_train_scaled = scaler.fit_transform(df_train)
df_test_scaled = scaler.transform(df_test)
df_valid_scaled = scaler.transform(df_valid)

# H2O AutoML

In [14]:
# initialize H2O
h2o.init(log_dir="h2o_logs", log_level="WARN")

# read as h2o file
h2o_train = h2o.H2OFrame(df_train_scaled)
h2o_valid = h2o.H2OFrame(df_valid_scaled)
h2o_test = h2o.H2OFrame(df_test_scaled)

# For binary classification, response should be a factor
h2o_train[target] = h2o_train[target].asfactor()
h2o_valid[target] = h2o_valid[target].asfactor()
h2o_test[target] = h2o_test[target].asfactor()

# Define AML task
aml = H2OAutoML(seed=random_state, max_runtime_secs=180)

# over/under sample for classification tasks
aml.balance_classes = True

# Run it
_ = aml.train(
    x=list(cont_cols),
    y=target,
    training_frame=h2o_train,
    leaderboard_frame=h2o_valid,
)

m = aml.get_best_model()

# Leaderboard, show and save
lb = h2o.automl.get_leaderboard(aml, extra_columns="ALL")
print(lb)
# save results
h2o.export_file(lb, path="h2o_logs/leaderboard.csv", force=True)

print(m.model_performance(h2o_valid))

# MOJO is h2o version agnostic
m.save_mojo("h2o_logs/bestmodel.zip")

predictions = m.predict(h2o_test)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,39 mins 03 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_palo_x2frqq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.419 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
10:12:00.102: _train param, Dropping bad and constant columns: [P_ref, V_DC, m_d, m_q]

███████████████████
10:12:52.949: _train param, Dropping bad and constant columns: [P_ref, V_DC, m_d, m_q]

█████████████████
10:13:43.919: _train param, Dropping bad and constant columns: [P_ref, V_DC, m_d, m_q]

███████████████████
10:14:37.193: _train param, Dropping unused columns: [P_ref, V_DC, m_d, m_q]

████████| (done) 100%
model_id                                                   mean_per_class_error      logloss        rmse          mse    training_time_ms    predict_time_per_row_ms  algo
XGBoost_1_AutoML_4_20230126_101200                                   0.00150103  0.000509257  0.0080350

In [16]:
print(lb)
print(m.model_performance(h2o_valid))

model_id                                                   mean_per_class_error      logloss        rmse          mse    training_time_ms    predict_time_per_row_ms  algo
XGBoost_1_AutoML_4_20230126_101200                                   0.00150103  0.000509257  0.00803503  6.45617e-05               52576                   0.007552  XGBoost
GBM_1_AutoML_4_20230126_101200                                       0.00309357  0.0181482    0.0531478   0.00282469                51257                   0.041447  GBM
StackedEnsemble_BestOfFamily_1_AutoML_4_20230126_101200              0.203173    0.0271085    0.0720747   0.00519477                23577                   0.009622  StackedEnsemble
GLM_1_AutoML_4_20230126_101200                                       0.459538    0.36961      0.355018    0.126038                  50907                   0.000476  GLM
[4 rows x 8 columns]

ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 6.456174218800017e-05
RMSE: 0.0080350321833

In [28]:
print("Classification report:\n{}".format(classification_report(h2o.as_list(predictions["predict"]), df_test[target])))

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23554
           1       1.00      1.00      1.00       317
           2       1.00      1.00      1.00      4002
           3       1.00      1.00      1.00      4002
           4       0.98      1.00      0.99       131

    accuracy                           1.00     32006
   macro avg       1.00      1.00      1.00     32006
weighted avg       1.00      1.00      1.00     32006



# LightGBM

In [7]:
from scipy.misc import derivative

def focal_loss_lgb(y_pred, dtrain, alpha, gamma, num_class):
    """
    Focal Loss for lightgbm

    Parameters:
    -----------
    y_pred: numpy.ndarray
        array with the predictions
    dtrain: lightgbm.Dataset
    alpha, gamma: float
        See original paper https://arxiv.org/pdf/1708.02002.pdf
    num_class: int
        number of classes
    """
    a,g = alpha, gamma
    y_true = dtrain.label
    # N observations x num_class arrays
    y_true = np.eye(num_class)[y_true.astype('int')]
    y_pred = y_pred.reshape(-1,num_class, order='F')
    # alpha and gamma multiplicative factors with BCEWithLogitsLoss
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    # flatten in column-major (Fortran-style) order
    return grad.flatten('F'), hess.flatten('F')

def focal_loss_lgb_eval_error(y_pred, dtrain, alpha, gamma, num_class):
    """
    Focal Loss for lightgbm

    Parameters:
    -----------
    y_pred: numpy.ndarray
        array with the predictions
    dtrain: lightgbm.Dataset
    alpha, gamma: float
        See original paper https://arxiv.org/pdf/1708.02002.pdf
    num_class: int
        number of classes
    """
    a,g = alpha, gamma
    y_true = dtrain.label
    y_true = np.eye(num_class)[y_true.astype('int')]
    y_pred = y_pred.reshape(-1, num_class, order='F')
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    # a variant can be np.sum(loss)/num_class
    return 'focal_loss', np.mean(loss), False

In [8]:
n_class = pd.concat([df_train_scaled, df_valid_scaled, df_test_scaled])[target].nunique()
custom = utils.LGBM_custom_score(n_class=n_class)
fobj = lambda preds, data: custom.lgbm_focal_loss(preds, data, 0.25, 1.0)
#feval = lambda preds, data: focal_loss_lgb_eval_error(preds, data, 0.25, 1.0, n_class)
feval = [
    lambda preds, data: [
        custom.lgbm_focal_loss_eval(preds, data, 0.25, 1.0),
         custom.lgbm_f1(preds, data),
         custom.lgbm_precision(preds, data),
         custom.lgbm_recall(preds, data),
         custom.lgbm_accuracy(preds, data),
    ]
]
# ray_metric = "-" + "focal_loss"
# ray_metric = "multi_logloss"

config = {}
config["objective"] = "multiclass"
config["num_classes"] = n_class

# config = {"verbose": -1}
# config["is_unbalance"] = True

In [9]:
df_train_scaled_enc = df_train_scaled.copy()
df_valid_scaled_enc = df_valid_scaled.copy()
df_test_scaled_enc = df_test_scaled.copy()
lgb_cat_cols = []

lgbtrain = lgbm.Dataset(
    df_train_scaled_enc.drop(columns=[target, identifier]),
    df_train_scaled_enc[target],
    categorical_feature=lgb_cat_cols,
    free_raw_data=False,
)
lgbvalid = lgbm.Dataset(
    df_valid_scaled_enc.drop(columns=[target, identifier]),
    df_valid_scaled_enc[target],
    reference=lgbtrain,
    free_raw_data=False,
)
# Final TRAIN/TEST
ftrain = pd.concat([df_train_scaled_enc, df_valid_scaled_enc]).reset_index(drop=True)
flgbtrain = lgbm.Dataset(
    ftrain.drop(columns=[target, identifier]),
    ftrain[target],
    categorical_feature=lgb_cat_cols,
    free_raw_data=False,
)
lgbtest = lgbm.Dataset(
    df_test_scaled_enc.drop(columns=[target, identifier]),
    df_test_scaled_enc[target],
    categorical_feature=lgb_cat_cols,
    reference=flgbtrain,
    free_raw_data=False,
)

In [10]:
model = lgbm.train(
    config,
    lgbtrain,
    valid_sets=[lgbvalid],
    valid_names=[""],
    fobj=fobj,
    feval=feval,
)

Using categorical_feature in Dataset.
categorical_feature in Dataset is overridden.
New categorical_feature is []


[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 256044, number of used features: 9
[LightGBM] [Warning] Using self-defined objective function


scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	's focal_loss: 0.195762	's f1: 0.999781	's precision: 0.999781	's recall: 0.999781	's accuracy: 0.999781


scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


...

[98]	's focal_loss: 0.00012123	's f1: 0.999875	's precision: 0.999875	's recall: 0.999875	's accuracy: 0.999875


scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


[99]	's focal_loss: 0.000121266	's f1: 0.999875	's precision: 0.999875	's recall: 0.999875	's accuracy: 0.999875


scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


[100]	's focal_loss: 0.000121365	's f1: 0.999875	's precision: 0.999875	's recall: 0.999875	's accuracy: 0.999875


In [11]:
res = model.predict(lgbtest.data).argmax(1)

result = pd.DataFrame(
    {
        "predicted": res,
        "ground_truth": df_test[target].values,
    }
)

print("Classification report:\n{}".format(classification_report(result["predicted"], result["ground_truth"])))

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23555
           1       1.00      1.00      1.00       317
           2       1.00      1.00      1.00      4002
           3       1.00      1.00      1.00      4000
           4       0.99      1.00      1.00       132

    accuracy                           1.00     32006
   macro avg       1.00      1.00      1.00     32006
weighted avg       1.00      1.00      1.00     32006



In [12]:
# shap_explainer = lightgbm_explainer.ShapExplainer()
# shap_explainer.fit(
#     model=model,
#     explainer_type="default",
# )
# shap_explainer.explain_decision_plot(
#     X_tab_explain=lgbtest.data[0].values,
#     feature_names=model.feature_name(),
# )

# Deep Learning Models

In [8]:
X_train, X_valid, tab_preprocessor = utils.dl_train_prep(
    data_train=df_train_scaled,
    data_valid=df_valid_scaled,
    identifier=identifier,
    cont_cols=cont_cols,
    target_col=target,
)

test_n_valid_combined = True
# X_tab_test = tab_preprocessor.transform(data_test_scaled).astype(float)
if test_n_valid_combined:
    test = pd.concat([df_valid_scaled, df_test_scaled]).reset_index(drop=True)
else:
    test = df_test_scaled.copy()

X_test = {"X_tab": tab_preprocessor.transform(test)}

n_classes = pd.concat([df_train_scaled, df_valid_scaled, df_test_scaled])[target].nunique()
metrics = utils.dl_metrics(n_classes)


input_layer = len(tab_preprocessor.continuous_cols)
output_layer = n_classes
hidden_layers = utils.dl_design(
    input_layer, 2, output_layer, design="funnel"
).hidden_layers()

Continuous columns will not be normalised


## Transformers

In [9]:
deeptabular = TabTransformer(input_dim=32, embed_continuous=True, column_idx=tab_preprocessor.column_idx, continuous_cols=tab_preprocessor.continuous_cols)
model = WideDeep(deeptabular=deeptabular, pred_dim=output_layer)

In [10]:
# Initializers/Optimizers/Schedulers/callbacks
initializers = {"deeptabular": XavierNormal}
deeptab_opt = SGD(model.deeptabular.parameters(), lr=0.001)
deeptab_sch = lr_scheduler.StepLR(deeptab_opt, step_size=5)
optimizers = {"deeptabular": deeptab_opt}
schedulers = {"deeptabular": deeptab_sch}

early_stopping = EarlyStopping()
model_checkpoint = ModelCheckpoint(save_best_only=True, verbose=1)

In [11]:
len(X_train["target"])

256044

In [12]:
objective = "multiclass_focal_loss"
dataloader = DataLoaderImbalanced

trainer = Trainer(
    model,
    objective=objective,
    callbacks=[early_stopping, model_checkpoint],
    lr_schedulers=schedulers,
    initializers=initializers,
    optimizers=optimizers,
    metrics=metrics,
)

trainer.fit(
    X_train=X_train,
    X_val=X_valid,
    n_epochs=5,
    batch_size=100,
    custom_dataloader=dataloader,
    oversample_mul=5,
)

  0%|          | 0/54 [00:00<?, ?it/s]

: 

: 

In [ ]:
actual = test[target]
predicted = trainer.predict(**X_test)
#predicted_mc = trainer.predict_uncertainty(**X_test, uncertainty_granularity=10)[:, -1],
classification_report(predicted, actual)

## w RayTune

In [ ]:
%%time
# Optimizers
deep_opt_sgd_01 = SGD(model.deeptabular.parameters(), lr=0.1)
deep_opt_sgd_001 = SGD(model.deeptabular.parameters(), lr=0.01)
deep_opt_adam_01 = Adam(model.deeptabular.parameters(), lr=0.1)
deep_opt_adam_001 = Adam(model.deeptabular.parameters(), lr=0.01)
# LR Schedulers
deep_sch_StepLR5 = lr_scheduler.StepLR(deep_opt, step_size=5)
deep_sch_StepLR10 = lr_scheduler.StepLR(deep_opt, step_size=10)

input_layer = len(tab_preprocessor.continuous_cols)
output_layer = n_classes

hidden_layers2 = utils.dl_design(input_layer, 2, output_layer, design="funnel")
hidden_layers3 = utils.dl_design(input_layer, 3, output_layer, design="funnel")
hidden_layers5 = utils.dl_design(input_layer, 5, output_layer, design="funnel")
hidden_layers10 = utils.dl_design(input_layer, 10, output_layer, design="funnel")

config = {
    "batch_size": tune.grid_search([100, 1000, 10000]),
    "deeptab_opt": tune.grid_search(
        [
            deep_opt_sgd_01,
            deep_opt_adam_01,
            deep_opt_sgd_001,
            deep_opt_adam_001,
        ]
    ),
    "deeptab_sch": tune.grid_search([deep_sch_StepLR5]),  # , deep_sch_StepLR10]),
    "hidden_layers": tune.grid_search(
        [hidden_layers2, hidden_layers3, hidden_layers5, hidden_layers10]
    ),
    "wandb": {
        "project": "dl_gm",
        "api_key_file": "/home/jovyan/repos/pltv/data/wandb_api.key",
    },
}

objective = "multiclass_focal_loss"
dataloader = DataLoaderImbalanced


@wandb_mixin
def training_function(config, X_train, X_val):
    early_stopping = EarlyStopping()
    model_checkpoint = ModelCheckpoint(save_best_only=True, wb=wandb)

    deeptabular = TabMlp(
        mlp_hidden_dims=config["hidden_layers"].hidden_layers(),
        column_idx=tab_preprocessor.column_idx,
        embed_input=tab_preprocessor.embeddings_input,
        continuous_cols=tab_preprocessor.continuous_cols,
        mlp_batchnorm=True,
        mlp_batchnorm_last=True,
        mlp_linear_first=True,
    )

    model = WideDeep(wide=wide, deeptabular=deeptabular)

    trainer = Trainer(
        model,
        objective=objective,
        callbacks=[RayTuneReporter, early_stopping, model_checkpoint],
        lr_schedulers={"deeptabular": config["deeptab_sch"]},
        initializers={"deeptabular": XavierNormal},
        optimizers={"deeptabular": config["deeptab_opt"]},
        metrics=metrics,
        verbose=0,
    )

    trainer.fit(
        X_train=X_train,
        X_val=X_val,
        n_epochs=50,
        batch_size=config["batch_size"],
        custom_dataloader=dataloader,
        oversample_mul=5,
    )


# https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-scheduler-hyperband
asha_scheduler = AsyncHyperBandScheduler(
    time_attr="training_iteration",
    metric="_metric/val_loss",
    mode="min",
    max_t=100,
    grace_period=10,
    reduction_factor=3,
    brackets=1,
)

analysis = tune.run(
    tune.with_parameters(training_function, X_train=X_train, X_val=X_valid),
    # resources_per_trial={"cpu": 4, "gpu": 0},
    num_samples=1,
    progress_reporter=JupyterNotebookReporter(overwrite=True),
    scheduler=asha_scheduler,
    config=config,
    callbacks=[
        WandbLoggerCallback(
            project=config["wandb"]["project"],
            api_key_file=config["wandb"]["api_key_file"],
            log_config=True,
        )
    ],
)

### Train the Best model

In [ ]:
%%time
params = copy(analysis.get_best_config("_metric/val_loss", "min"))
params.pop("wandb")

trainer = Trainer(
    model,
    objective=objective,
    callbacks=[LRHistory(n_epochs=10)],
    lr_schedulers={"wide": params["wide_sch"], "deeptabular": params["deeptab_sch"]},
    initializers={"wide": XavierNormal, "deeptabular": XavierNormal},
    optimizers={"wide": params["wide_opt"], "deeptabular": params["deeptab_opt"]},
    metrics=metrics,
    verbose=0,
)

trainer.fit(
    X_train=X_train,
    X_val=X_val,
    n_epochs=5,
    batch_size=params["batch_size"],
    custom_dataloader=dataloader,
    oversample_mul=5,
)